In [1]:
# Barriers


import threading
import time
import random

class myThread(threading.Thread):

    def __init__(self, barrier):
      threading.Thread.__init__(self)
      self.barrier = barrier;

    def run(self):
      print("Thread {} working on something".format(threading.current_thread()))
      time.sleep(random.randint(1,10))
      print("Thread {} is joining {} waiting on Barrier".format(threading.current_thread(), self.barrier.n_waiting))
      self.barrier.wait()
      
      print("Barrier has been lifted, continuing with work")


barrier = threading.Barrier(4)

threads = []

for i in range(4):
  thread = myThread(barrier)
  thread.start()
  threads.append(thread)

for t in threads:
    t.join()

Thread <myThread(Thread-4, started 140084376233728)> working on something
Thread <myThread(Thread-5, started 140084367841024)> working on something
Thread <myThread(Thread-6, started 140084359448320)> working on something
Thread <myThread(Thread-7, started 140084351055616)> working on something
Thread <myThread(Thread-5, started 140084367841024)> is joining 0 waiting on Barrier
Thread <myThread(Thread-7, started 140084351055616)> is joining 1 waiting on Barrier
Thread <myThread(Thread-4, started 140084376233728)> is joining 2 waiting on Barrier
Thread <myThread(Thread-6, started 140084359448320)> is joining 3 waiting on Barrier
Barrier has been lifted, continuing with work
Barrier has been lifted, continuing with work
Barrier has been lifted, continuing with work
Barrier has been lifted, continuing with work


In [2]:
# Thread Join

import threading
import time

def ourThread(i):
  print("Thread {} Started".format(i))
  time.sleep(i*2)
  print("Thread {} Finished".format(i))

def main():
  thread = threading.Thread(target=ourThread, args=(1,))
  thread.start()

  print("Is thread 1 Finished?")

  thread2 = threading.Thread(target=ourThread, args=(2,))
  thread2.start()
  thread2.join()

  print("Thread 2 definitely finished")

if __name__ == '__main__':
  main()

Thread 1 Started
Is thread 1 Finished?
Thread 2 Started
Thread 1 Finished
Thread 2 Finished
Thread 2 definitely finished


In [3]:
# Events


import threading
import time

def myThread(myEvent):
  while not myEvent.is_set():
    print("Waiting for Event to be set")
    time.sleep(1)
  print("myEvent has been set")

def main():
  myEvent = threading.Event()
  thread1 = threading.Thread(target=myThread, args=(myEvent,))
  thread1.start()

  time.sleep(10)
  myEvent.set()


if __name__ == '__main__':
  main()

Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set
Waiting for Event to be set


In [4]:
# Locks


import threading
import time
import random

counter = 1
lock = threading.Lock()

def workerA():
  global counter
  lock.acquire()
  try:
    while counter < 1000:
      counter += 1
      print("Worker A is incrementing counter to {}".format(counter))
      
  finally:
    lock.release()

def workerB():
  global counter
  lock.acquire()
  try:
    while counter > -1000:
      counter -= 1
      print("Worker B is decrementing counter to {}".format(counter))
      
  finally:
    lock.release()

def main():
  t0 = time.time()
  thread1 = threading.Thread(target=workerA)
  thread2 = threading.Thread(target=workerB)

  thread1.start()
  thread2.start()

  thread1.join()
  thread2.join()

  t1 = time.time()

  print("Execution Time {}".format(t1-t0))

if __name__ == '__main__':
  main()



myEvent has been set
Worker A is incrementing counter to 2
Worker A is incrementing counter to 3
Worker A is incrementing counter to 4
Worker A is incrementing counter to 5
Worker A is incrementing counter to 6
Worker A is incrementing counter to 7
Worker A is incrementing counter to 8
Worker A is incrementing counter to 9
Worker A is incrementing counter to 10
Worker A is incrementing counter to 11
Worker A is incrementing counter to 12
Worker A is incrementing counter to 13
Worker A is incrementing counter to 14
Worker A is incrementing counter to 15
Worker A is incrementing counter to 16
Worker A is incrementing counter to 17
Worker A is incrementing counter to 18
Worker A is incrementing counter to 19
Worker A is incrementing counter to 20
Worker A is incrementing counter to 21
Worker A is incrementing counter to 22
Worker A is incrementing counter to 23
Worker A is incrementing counter to 24
Worker A is incrementing counter to 25
Worker A is incrementing counter to 26
Worker A is 

Worker B is decrementing counter to -501
Worker B is decrementing counter to -502
Worker B is decrementing counter to -503
Worker B is decrementing counter to -504
Worker B is decrementing counter to -505
Worker B is decrementing counter to -506
Worker B is decrementing counter to -507
Worker B is decrementing counter to -508
Worker B is decrementing counter to -509
Worker B is decrementing counter to -510
Worker B is decrementing counter to -511
Worker B is decrementing counter to -512
Worker B is decrementing counter to -513
Worker B is decrementing counter to -514
Worker B is decrementing counter to -515
Worker B is decrementing counter to -516
Worker B is decrementing counter to -517
Worker B is decrementing counter to -518
Worker B is decrementing counter to -519
Worker B is decrementing counter to -520
Worker B is decrementing counter to -521
Worker B is decrementing counter to -522
Worker B is decrementing counter to -523
Worker B is decrementing counter to -524
Worker B is decr

In [8]:
#ThreadSafe Locks


from threading import Lock

def lock_class(methodnames, lockfactory):
    return lambda cls: make_threadsafe(cls, methodnames, lockfactory)

def lock_method(method):
    if getattr(method, '__is_locked', False):
        raise TypeError("Method %r is already locked!" % method)
    def locked_method(self, *arg, **kwarg):
        with self._lock:
            return method(self, *arg, **kwarg)
    locked_method.__name__ = '%s(%s)' % ('lock_method', method.__name__)
    locked_method.__is_locked = True
    return locked_method


def make_threadsafe(cls, methodnames, lockfactory):
    init = cls.__init__
    def newinit(self, *arg, **kwarg):
        init(self, *arg, **kwarg)
        self._lock = lockfactory()
    cls.__init__ = newinit

    for methodname in methodnames:
        oldmethod = getattr(cls, methodname)
        newmethod = lock_method(oldmethod)
        setattr(cls, methodname, newmethod)

    return cls


@lock_class(['add','remove'], Lock)
class ClassDecoratorLockedSet(set):

    @lock_method # if you double-lock a method, a TypeError is raised
    def lockedMethod(self):
        print("This section of our code would be thread safe")
        pass


In [26]:
#LockSets

class LockedSet(set):
    """A set where add(), remove(), and 'in' operator are thread-safe"""

    def __init__(self, *args, **kwargs):
        self._lock = Lock()
        super(LockedSet, self).__init__(*args, **kwargs)

    def add(self, elem):
        with self._lock:
            super(LockedSet, self).add(elem)

    def remove(self, elem):
        with self._lock:
            super(LockedSet, self).remove(elem)

    def __contains__(self, elem):
        with self._lock:
            super(LockedSet, self).__contains__(elem)

condition acquired by Thread-24
89 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
89 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
192 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
192 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 994
Condition Released by Publisher: Thread-26
994 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
15 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
15 popped from list by Thread-25
condition released by Thread-25
condition acq

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 880
Condition Released by Publisher: Thread-26
880 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
36 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
36 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
152 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
152 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 646
Condition Released by Publisher: Thread-26
646 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Cond

condition acquired by Thread-24
179 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
179 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
255 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
255 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 752
Condition Released by Publisher: Thread-26
752 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
78 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
78 popped from list by Thread-25
condition released by Thread-25
condition a

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 877
Condition Released by Publisher: Thread-26
877 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
79 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
79 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
44 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
44 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 11
Condition Released by Publisher: Thread-26
11 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Conditio

condition acquired by Thread-24
155 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
155 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
43 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
43 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 917
Condition Released by Publisher: Thread-26
917 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
167 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
167 popped from list by Thread-25
condition released by Thread-25
condition a

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 709
Condition Released by Publisher: Thread-26
709 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
236 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
236 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
194 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
194 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 406
Condition Released by Publisher: Thread-26
406 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Co

condition acquired by Thread-24
191 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
191 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
114 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
114 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 371
Condition Released by Publisher: Thread-26
371 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
243 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
243 popped from list by Thread-25
condition released by Thread-25
condition

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 426
Condition Released by Publisher: Thread-26
426 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
215 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
215 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
248 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
248 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 190
Condition Released by Publisher: Thread-26
190 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Co

In [6]:
# RLock

import threading
import time

class myWorker():

  def __init__(self):
    self.a = 1
    self.b = 2
    self.rlock = threading.RLock()
  
  def modifyA(self):
    with self.rlock:
      print("Modifying A : RLock Acquired: {}".format(self.rlock._is_owned()))
      print("{}".format(self.rlock))
      self.a = self.a + 1
      time.sleep(5)

  def modifyB(self):
    with self.rlock:
      print("Modifying B : RLock Acquired: {}".format(self.rlock._is_owned()))
      print("{}".format(self.rlock))
      self.b = self.b - 1
      time.sleep(5)

  def modifyBoth(self):
    with self.rlock:
      print("Rlock acquired, modifying A and B")
      print("{}".format(self.rlock))
      self.modifyA()
      print("{}".format(self.rlock))
      self.modifyB()
    print("{}".format(self.rlock))


workerA = myWorker()
workerA.modifyBoth()


Rlock acquired, modifying A and B
<locked _thread.RLock object owner=140084954908480 count=1 at 0x7f67ed7a4630>
Modifying A : RLock Acquired: True
<locked _thread.RLock object owner=140084954908480 count=2 at 0x7f67ed7a4630>
<locked _thread.RLock object owner=140084954908480 count=1 at 0x7f67ed7a4630>
Modifying B : RLock Acquired: True
<locked _thread.RLock object owner=140084954908480 count=2 at 0x7f67ed7a4630>
<unlocked _thread.RLock object owner=0 count=0 at 0x7f67ed7a4630>


In [7]:
# Rlock Vs Lock


import threading
import time

class myWorker():

  def __init__(self):
    self.a = 1
    self.b = 2
    self.lock = threading.Lock()
  
  def modifyA(self):
    print("Modifying A : RLock Acquired: {}".format(self.lock._is_owned()))
    print("{}".format(self.lock))
    self.a = self.a + 1
    time.sleep(5)

  def modifyB(self):
    print("Modifying B : Lock Acquired: {}".format(self.lock._is_owned()))
    print("{}".format(self.lock))
    self.b = self.b - 1
    time.sleep(5)

  def modifyBoth(self):
    with self.lock:
      print("lock acquired, modifying A and B")
      print("{}".format(self.lock))
      self.modifyA()
      print("{}".format(self.lock))
      self.modifyB()
    print("{}".format(self.lock))


workerA = myWorker()
workerA.modifyBoth()


lock acquired, modifying A and B
<locked _thread.lock object at 0x7f67efffe058>


AttributeError: '_thread.lock' object has no attribute '_is_owned'

In [9]:
# Semaphores


import threading
import time
import random

class TicketSeller(threading.Thread):
  ticketsSold = 0

  def __init__(self, semaphore):
    threading.Thread.__init__(self);
    self.sem = semaphore
    print("Ticket Seller Started Work")

  def run(self):
    global ticketsAvailable
    running = True
    while running:
      self.randomDelay()
      
      self.sem.acquire()
      if(ticketsAvailable <= 0):
        running = False
      else:
        self.ticketsSold = self.ticketsSold + 1
        ticketsAvailable = ticketsAvailable - 1
        print("{} Sold One ({} left)".format(self.getName(), ticketsAvailable))
      self.sem.release()
    print("Ticket Seller {} Sold {} tickets in total".format(self.getName(), self.ticketsSold))

  def randomDelay(self):
    time.sleep(random.randint(0,4)/4)


# our sempahore primitive
semaphore = threading.Semaphore()
# Our Ticket Allocation
ticketsAvailable = 200

# our array of sellers
sellers = []
for i in range(4):
  seller = TicketSeller(semaphore)
  seller.start()
  sellers.append(seller)

# joining all our sellers
for seller in sellers:
  seller.join()


Ticket Seller Started Work
Ticket Seller Started Work
Thread-15 Sold One (199 left)
Thread-15 Sold One (198 left)
Ticket Seller Started Work
Ticket Seller Started Work
Thread-14 Sold One (197 left)
Thread-14 Sold One (196 left)
Thread-17 Sold One (195 left)
Thread-14 Sold One (194 left)
Thread-14 Sold One (193 left)
Thread-14 Sold One (192 left)
Thread-14 Sold One (191 left)
Thread-17 Sold One (190 left)
Thread-17 Sold One (189 left)
Thread-15 Sold One (188 left)
Thread-16 Sold One (187 left)
Thread-14 Sold One (186 left)
Thread-17 Sold One (185 left)
Thread-16 Sold One (184 left)
Thread-14 Sold One (183 left)
Thread-16 Sold One (182 left)
Thread-17 Sold One (181 left)
Thread-17 Sold One (180 left)
Thread-14 Sold One (179 left)
Thread-17 Sold One (178 left)
Thread-15 Sold One (177 left)
Thread-14 Sold One (176 left)
Thread-17 Sold One (175 left)
Thread-17 Sold One (174 left)
Thread-16 Sold One (173 left)
Thread-15 Sold One (172 left)
Thread-14 Sold One (171 left)
Thread-17 Sold One (17

In [10]:
#Bounded Semaphores

import threading
import time
import random

class TicketSeller(threading.Thread):
  ticketsSold = 0

  def __init__(self, semaphore):
    threading.Thread.__init__(self);
    self.sem = semaphore
    print("Ticket Seller Started Work")

  def run(self):
    global ticketsAvailable
    running = True
    while running:
      self.randomDelay()
      
      self.sem.acquire()
      if(ticketsAvailable <= 0):
        running = False
      else:
        self.ticketsSold = self.ticketsSold + 1
        ticketsAvailable = ticketsAvailable - 1
        print("{} Sold One ({} left)".format(self.getName(), ticketsAvailable))
      self.sem.release()
    print("Ticket Seller {} Sold {} tickets in total".format(self.getName(), self.ticketsSold))

  def randomDelay(self):
    time.sleep(random.randint(0,4)/4)


# our sempahore primitive
semaphore = threading.BoundedSemaphore(2)
# Our Ticket Allocation
ticketsAvailable = 200

# our array of sellers
sellers = []
for i in range(4):
  seller = TicketSeller(semaphore)
  seller.start()
  sellers.append(seller)

# joining all our sellers
for seller in sellers:
  seller.join()


Ticket Seller Started Work
Ticket Seller Started Work
Ticket Seller Started Work
Ticket Seller Started Work
Thread-21 Sold One (199 left)
Thread-18 Sold One (198 left)
Thread-19 Sold One (197 left)
Thread-19 Sold One (196 left)
Thread-18 Sold One (195 left)
Thread-20 Sold One (194 left)
Thread-21 Sold One (193 left)
Thread-19 Sold One (192 left)
Thread-19 Sold One (191 left)
Thread-19 Sold One (190 left)
Thread-21 Sold One (189 left)
Thread-20 Sold One (188 left)
Thread-19 Sold One (187 left)
Thread-18 Sold One (186 left)
Thread-19 Sold One (185 left)
Thread-20 Sold One (184 left)
Thread-21 Sold One (183 left)
Thread-19 Sold One (182 left)
Thread-18 Sold One (181 left)
Thread-21 Sold One (180 left)
Thread-19 Sold One (179 left)
Thread-18 Sold One (178 left)
Thread-21 Sold One (177 left)
Thread-19 Sold One (176 left)
Thread-20 Sold One (175 left)
Thread-20 Sold One (174 left)
Thread-21 Sold One (173 left)
Thread-18 Sold One (172 left)
Thread-21 Sold One (171 left)
Thread-18 Sold One (17

In [11]:
# fork rlock join 



import threading
import time
import random

class Philosopher(threading.Thread):

  def __init__(self, name, leftFork, rightFork):
    print("{} Has Sat Down At the Table".format(name))
    threading.Thread.__init__(self, name=name)
    self.leftFork = leftFork
    self.rightFork = rightFork

  def run(self):
    print("{} has started thinking".format(threading.currentThread().getName()))
    while True:
      time.sleep(random.randint(1,5))
      print("{} has finished thinking".format(threading.currentThread().getName()))
      self.leftFork.acquire()
      time.sleep(random.randint(1,5))
      try:
        print("{} has acquired the left fork".format(threading.currentThread().getName()))

        self.rightFork.acquire()
        try:
          print("{} has attained both forks, currently eating".format(threading.currentThread().getName()))
        finally:
          self.rightFork.release()   
          print("{} has released the right fork".format(threading.currentThread().getName()))
      finally:
        self.leftFork.release()
        print("{} has released the left fork".format(threading.currentThread().getName()))

fork1 = threading.RLock()
fork2 = threading.RLock()
fork3 = threading.RLock()
fork4 = threading.RLock()
fork5 = threading.RLock()

philosopher1 = Philosopher("Kant", fork1, fork2)
philosopher2 = Philosopher("Aristotle", fork2, fork3)
philosopher3 = Philosopher("Spinoza", fork3, fork4)
philosopher4 = Philosopher("Marx", fork4, fork5)
philosopher5 = Philosopher("Russell", fork5, fork1)

philosopher1.start()
philosopher2.start()
philosopher3.start()
philosopher4.start()
philosopher5.start()

philosopher1.join()
philosopher2.join()
philosopher3.join()
philosopher4.join()
philosopher5.join()

Kant Has Sat Down At the Table
Aristotle Has Sat Down At the Table
Spinoza Has Sat Down At the Table
Marx Has Sat Down At the Table
Russell Has Sat Down At the Table
Kant has started thinking
Aristotle has started thinking
Spinoza has started thinking
Marx has started thinking
Russell has started thinking
Spinoza has finished thinking
Marx has finished thinking
Kant has finished thinking
Marx has acquired the left fork
Marx has attained both forks, currently eating
Marx has released the right fork
Marx has released the left fork
Kant has acquired the left fork
Kant has attained both forks, currently eating
Kant has released the right fork
Kant has released the left fork
Aristotle has finished thinking
Spinoza has acquired the left fork
Spinoza has attained both forks, currently eating
Spinoza has released the right fork
Spinoza has released the left fork
Russell has finished thinking
Kant has finished thinking
Marx has finished thinking
Aristotle has acquired the left fork
Aristotle ha

KeyboardInterrupt: 

In [13]:
# Producer Consumer 

import threading
import random
import time

class Producer(threading.Thread):
    """
    Produces random integers to a list
    """

    def __init__(self, integers, condition):
        """
        Constructor.

        @param integers list of integers
        @param condition condition synchronization object
        """
        threading.Thread.__init__(self)
        self.integers = integers
        self.condition = condition
    
    def run(self):
        """
        Thread run method. Append random integers to the integers list
        at random time.
        """
        while True:
            integer = random.randint(0, 256)
            self.condition.acquire()
            print('condition acquired by %s' % self.name)
            self.integers.append(integer) 
            print('%d appended to list by %s' % (integer, self.name))
            print('condition notified by %s' % self.name)
            self.condition.notify()
            print('condition released by %s' % self.name)
            self.condition.release()
            time.sleep(1)

class Consumer(threading.Thread):
    """
    Consumes random integers from a list
    """

    def __init__(self, integers, condition):
        """
        Constructor.

        @param integers list of integers
        @param condition condition synchronization object
        """
        threading.Thread.__init__(self)
        self.integers = integers
        self.condition = condition
    
    def run(self):
        """
        Thread run method. Consumes integers from list
        """
        while True:
            self.condition.acquire()
            print('condition acquired by %s' % self.name)
            while True:
                if self.integers:
                    integer = self.integers.pop()
                    print('%d popped from list by %s' % (integer, self.name))
                    break
                print('condition wait by %s' % self.name)
                self.condition.wait()
            print('condition released by %s' % self.name)
            self.condition.release()

def main():
    integers = []
    condition = threading.Condition()
    t1 = Producer(integers, condition)
    t2 = Consumer(integers, condition)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

if __name__ == '__main__':
    main()

condition acquired by Thread-24
99 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
condition acquired by Thread-25
99 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
157 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
157 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
condition acquired by Thread-24
132 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
132 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
13 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
13 popped from list by Thread-23
condition released by Thread-23
conditio

condition acquired by Thread-24
107 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
107 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
121 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
121 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
condition acquired by Thread-24
23 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
23 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
136 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
136 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condit

condition acquired by Thread-24
139 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
139 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
52 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
52 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
condition acquired by Thread-24
161 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
161 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
20 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
20 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
conditio

KeyboardInterrupt: 

condition acquired by Thread-24
143 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
143 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
248 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
248 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
condition acquired by Thread-24
1 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
1 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
181 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
181 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
conditio

In [14]:
# Publisher and Subscriber 


import threading
import random
import time

class Publisher(threading.Thread):

  def __init__(self, integers, condition):
    self.condition = condition
    self.integers = integers
    threading.Thread.__init__(self)

  def run(self):
    while True:
      integer = random.randint(0,1000)
      self.condition.acquire()
      print("Condition Acquired by Publisher: {}".format(self.name))
      self.integers.append(integer)
      print("Publisher {} appending to array: {}".format(self.name, integer))
      self.condition.notify()
      print("Condition Released by Publisher: {}".format(self.name))
      self.condition.release()
      time.sleep(1)

class Subscriber(threading.Thread):

  def __init__(self, integers, condition):
    self.integers = integers
    self.condition = condition
    threading.Thread.__init__(self)

  def run(self):
    while True:
      self.condition.acquire()
      print("Condition Acquired by Consumer: {}".format(self.name))
      while True:
        if self.integers:
          integer = self.integers.pop()
          print("{} Popped from list by Consumer: {}".format(integer, self.name))
          break
        print("Condition Wait by {}".format(self.name))
        self.condition.wait()
      print("Consumer {} Releasing Condition".format(self.name))
      self.condition.release()

def main():
  integers = []
  condition = threading.Condition()
  
  # Our Publisher
  pub1 = Publisher(integers, condition)
  pub1.start()

  # Our Subscribers
  sub1 = Subscriber(integers, condition)
  sub2 = Subscriber(integers, condition)
  sub1.start()
  sub2.start()

  ## Joining our Threads
  pub1.join()
  consumer1.join()
  consumer2.join()

if __name__ == '__main__':
  main()


Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 643
Condition Released by Publisher: Thread-26
Condition Acquired by Consumer: Thread-27
643 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
195 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
195 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
195 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
195 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 854
Condit

condition acquired by Thread-24
127 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
127 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
187 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
187 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 962
Condition Released by Publisher: Thread-26
962 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
178 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
178 popped from list by Thread-25
condition released by Thread-25
condition

KeyboardInterrupt: 

condition acquired by Thread-24
134 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
134 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
113 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
113 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23


In [15]:
# Queue 

import threading
import queue
import random
import time

def mySubscriber(queue):
  while not queue.empty():
    item = queue.get()
    if item is None:
      break
    print("{} removed {} from the queue".format(threading.current_thread(), item))
    queue.task_done()
    time.sleep(1)

myQueue = queue.Queue()
for i in range(10):
  myQueue.put(i)

print("Queue Populated")

threads = []
for i in range(4):
  thread = threading.Thread(target=mySubscriber, args=(myQueue,))
  thread.start()
  threads.append(thread)

for thread in threads:
  thread.join()



Queue Populated
<Thread(Thread-29, started 140083361470208)> removed 0 from the queue
<Thread(Thread-30, started 140083353077504)> removed 1 from the queue
<Thread(Thread-31, started 140083344684800)> removed 2 from the queue
<Thread(Thread-32, started 140083336292096)> removed 3 from the queue
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 562
Condition Released by Publisher: Thread-26
562 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
150 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
150 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
15 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
15 popped from list by Thread-23
con

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 228
Condition Released by Publisher: Thread-26
228 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
219 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
219 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
144 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
144 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23


In [16]:
# Queue Operations

import threading
import queue
import random
import time

def mySubscriber(queue):
  while True:
    item = queue.get()
    if item is None:
      break
    print("{} removed {} from the queue".format(threading.current_thread(), item))
    print("Queue Size is now: {}".format(queue.qsize()))
    queue.task_done()

myQueue = queue.Queue()
for i in range(10):
  myQueue.put(i)

print("Queue Populated")

threads = []
for i in range(4):
  thread = threading.Thread(target=mySubscriber, args=(myQueue,))
  thread.start()
  threads.append(thread)

myQueue.join()



Queue Populated
<Thread(Thread-33, started 140083353077504)> removed 0 from the queue
Queue Size is now: 9
<Thread(Thread-33, started 140083353077504)> removed 1 from the queue
Queue Size is now: 8
<Thread(Thread-33, started 140083353077504)> removed 2 from the queue
Queue Size is now: 7
<Thread(Thread-33, started 140083353077504)> removed 3 from the queue
Queue Size is now: 6
<Thread(Thread-33, started 140083353077504)> removed 4 from the queue
Queue Size is now: 5
<Thread(Thread-33, started 140083353077504)> removed 5 from the queue
Queue Size is now: 4
<Thread(Thread-33, started 140083353077504)> removed 6 from the queue
Queue Size is now: 3
<Thread(Thread-33, started 140083353077504)> removed 7 from the queue
Queue Size is now: 2
<Thread(Thread-33, started 140083353077504)> removed 8 from the queue
Queue Size is now: 1
<Thread(Thread-33, started 140083353077504)> removed 9 from the queue
Queue Size is now: 0
condition acquired by Thread-24
224 appended to list by Thread-24
conditio

In [17]:
# Queue Join


import threading
import queue
import time

def mySubscriber(queue):
  time.sleep(1)
  while not queue.empty():
    item = queue.get()
    if item is None:
      break
    print("{} removed {} from the queue".format(threading.current_thread(), item))
    queue.task_done()

myQueue = queue.Queue()
for i in range(5):
  myQueue.put(i)

print("Queue Populated")

thread = threading.Thread(target=mySubscriber, args=(myQueue,))
thread.start()

print("Not Progressing Till Queue is Empty")
myQueue.join()
print("Queue is now empty")




Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 264
Condition Released by Publisher: Thread-26
264 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
Queue Populated
Not Progressing Till Queue is Empty
condition acquired by Thread-24
4 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
4 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
145 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
145 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 478
Condition Released by Publisher: Thread-26
478 Popped from list by Con

In [18]:
# Priority Queue


import threading
import queue
import random
import time

def mySubscriber(queue):
  while not queue.empty():
    item = queue.get()
    if item is None:
      break
    print("{} removed {} from the queue".format(threading.current_thread(), item))
    queue.task_done()
    time.sleep(1)


myQueue = queue.PriorityQueue()

for i in range(5):
  myQueue.put(i, i)

for i in range(5):
  myQueue.put(i, i)

print("Queue Populated")

threads = []
for i in range(2):
  thread = threading.Thread(target=mySubscriber, args=(myQueue,))
  thread.start()
  threads.append(thread)

for thread in threads:
  thread.join()

print("Queue is empty")


Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 48
Condition Released by Publisher: Thread-26
48 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
Queue Populated
<Thread(Thread-38, started 140083327899392)> removed 0 from the queue
<Thread(Thread-39, started 140082841384704)> removed 0 from the queue
condition acquired by Thread-24
71 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
71 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
163 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
163 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher 

In [19]:
# Lifo Queues


import threading
import queue
import random
import time

def mySubscriber(queue):
  while not queue.empty():
    item = queue.get()
    if item is None:
      break
    print("{} removed {} from the queue".format(threading.current_thread(), item))
    queue.task_done()


myQueue = queue.LifoQueue()

for i in range(10):
  myQueue.put(i)

print("Queue Populated")

threads = []
for i in range(2):
  thread = threading.Thread(target=mySubscriber, args=(myQueue,))
  thread.start()
  threads.append(thread)

for thread in threads:
  thread.join()

print("Queue is empty")


condition acquired by Thread-24
123 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
123 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
Queue Populated
<Thread(Thread-40, started 140082841384704)> removed 9 from the queue
<Thread(Thread-40, started 140082841384704)> removed 8 from the queue<Thread(Thread-41, started 140083327899392)> removed 7 from the queue
<Thread(Thread-41, started 140083327899392)> removed 6 from the queue
<Thread(Thread-41, started 140083327899392)> removed 5 from the queue
<Thread(Thread-41, started 140083327899392)> removed 4 from the queue
<Thread(Thread-41, started 140083327899392)> removed 3 from the queue
<Thread(Thread-41, started 140083327899392)> removed 2 from the queue
<Thread(Thread-41, started 140083327899392)> removed 1 from the queue
<Thread(Thread-41, started 140083327899392)> removed 0 from the queue

Queue is empty
condition acq

In [20]:
# Full Queue



import threading
import queue
import time

def myPublisher(queue):
  while not queue.full():
    queue.put(1)
    print("{} Appended 1 to queue: {}".format(threading.current_thread(), queue.qsize()))
    time.sleep(1)

myQueue = queue.Queue()
for i in range(10):
  myQueue.put(i)

threads = []
for i in range(4):
  thread = threading.Thread(target=mySubscriber, args=(myQueue,))
  thread.start()
  threads.append(thread)

myQueue.join





<Thread(Thread-42, started 140082841384704)> removed 0 from the queue
<Thread(Thread-42, started 140082841384704)> removed 1 from the queue
<Thread(Thread-42, started 140082841384704)> removed 2 from the queue
<Thread(Thread-42, started 140082841384704)> removed 3 from the queue
<Thread(Thread-42, started 140082841384704)> removed 4 from the queue
<Thread(Thread-42, started 140082841384704)> removed 5 from the queue
<Thread(Thread-42, started 140082841384704)> removed 6 from the queue
<Thread(Thread-42, started 140082841384704)> removed 7 from the queue
<Thread(Thread-42, started 140082841384704)> removed 8 from the queue
<Thread(Thread-42, started 140082841384704)> removed 9 from the queue


<bound method Queue.join of <queue.Queue object at 0x7f67ec6d4da0>>

condition acquired by Thread-24
16 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
16 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
44 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
44 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 188
Condition Released by Publisher: Thread-26
188 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
191 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
191 popped from list by Thread-25
condition released by Thread-25
condition acq

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 910
Condition Released by Publisher: Thread-26
910 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28


In [21]:
# Deque


import collections

doubleEndedQueue = collections.deque('123456')

print("Dequeue: {}".format(doubleEndedQueue))

for item in doubleEndedQueue:
  print("Item {}".format(item))

print("Left Most Element: {}".format(doubleEndedQueue[0]))
print("Right Most Element: {}".format(doubleEndedQueue[-1]))

doubleEndedQueue.remove('1')
print("Removing Element: {}".format(doubleEndedQueue))

Dequeue: deque(['1', '2', '3', '4', '5', '6'])
Item 1
Item 2
Item 3
Item 4
Item 5
Item 6
Left Most Element: 1
Right Most Element: 6
Removing Element: deque(['2', '3', '4', '5', '6'])
condition acquired by Thread-24
85 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
85 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
91 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
91 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 467
Condition Released by Publisher: Thread-26
467 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thre

In [22]:
# Insert Deque


import collections

doubleEndedQueue = collections.deque('123456')

print("Deque: {}".format(doubleEndedQueue))

doubleEndedQueue.insert(5,5)

print("Deque: {}".format(doubleEndedQueue))




Deque: deque(['1', '2', '3', '4', '5', '6'])
Deque: deque(['1', '2', '3', '4', '5', 5, '6'])
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 78
Condition Released by Publisher: Thread-26
78 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
255 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
255 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
78 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
78 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 726
Condition Released by Publish

In [23]:
# Append Deque


import collections

doubleEndedQueue = collections.deque('123456')

print("Deque: {}".format(doubleEndedQueue))

# Removing Elements from our arra
doubleEndedQueue.append('1')
print("Deque: {}".format(doubleEndedQueue))

doubleEndedQueue.appendleft('6')
print("Deque: {}".format(doubleEndedQueue))




Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 14
Condition Released by Publisher: Thread-26
14 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
Deque: deque(['1', '2', '3', '4', '5', '6'])
Deque: deque(['1', '2', '3', '4', '5', '6', '1'])
Deque: deque(['6', '1', '2', '3', '4', '5', '6', '1'])
condition acquired by Thread-24
242 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
242 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
213 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
213 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thre

In [24]:
# Rotate Deque


import collections

doubleEndedQueue = collections.deque('123456')

print("Deque: {}".format(doubleEndedQueue))

doubleEndedQueue.rotate(3)

print("Deque: {}".format(doubleEndedQueue))

doubleEndedQueue.rotate(-2)

print("Deque {}".format(doubleEndedQueue))

Deque: deque(['1', '2', '3', '4', '5', '6'])
Deque: deque(['4', '5', '6', '1', '2', '3'])
Deque deque(['6', '1', '2', '3', '4', '5'])
condition acquired by Thread-24
168 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
168 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
246 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
246 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 250
Condition Released by Publisher: Thread-26
250 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
111 appended to list by Thread-24
condi

In [25]:
# Remove Deque


import collections

doubleEndedQueue = collections.deque('123456')

print("Deque: {}".format(doubleEndedQueue))

# Removing Elements from our arra
rightPop = doubleEndedQueue.pop()
print(rightPop)
print("Deque: {}".format(doubleEndedQueue))

leftPop = doubleEndedQueue.popleft()
print(leftPop)
print("Deque: {}".format(doubleEndedQueue))

Deque: deque(['1', '2', '3', '4', '5', '6'])
6
Deque: deque(['1', '2', '3', '4', '5'])
1
Deque: deque(['2', '3', '4', '5'])
condition acquired by Thread-24
126 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
126 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
56 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
56 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 17
Condition Released by Publisher: Thread-26
17 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
4 appended to list by Thread-24
condition notified by

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 653
Condition Released by Publisher: Thread-26
653 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
13 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
13 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
82 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
82 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 388
Condition Released by Publisher: Thread-26
388 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condit

condition acquired by Thread-24
16 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
16 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
168 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
168 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 499
Condition Released by Publisher: Thread-26
499 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
183 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
183 popped from list by Thread-25
condition released by Thread-25
condition a

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 690
Condition Released by Publisher: Thread-26
690 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
186 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
186 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
207 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
207 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 110
Condition Released by Publisher: Thread-26
110 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Co

condition acquired by Thread-24
211 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
211 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
160 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
160 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 830
Condition Released by Publisher: Thread-26
830 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
189 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
189 popped from list by Thread-25
condition released by Thread-25
condition

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 472
Condition Released by Publisher: Thread-26
472 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
172 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
172 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
142 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
142 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 935
Condition Released by Publisher: Thread-26
935 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Co

condition acquired by Thread-24
96 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
96 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
115 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
115 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 466
Condition Released by Publisher: Thread-26
466 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
207 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
207 popped from list by Thread-25
condition released by Thread-25
condition a

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 220
Condition Released by Publisher: Thread-26
220 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
184 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
184 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
16 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
16 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 295
Condition Released by Publisher: Thread-26
295 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Cond

In [27]:
#Class Decorator


from threading import Lock

def lock_class(methodnames, lockfactory):
    return lambda cls: make_threadsafe(cls, methodnames, lockfactory)

def lock_method(method):
    if getattr(method, '__is_locked', False):
        raise TypeError("Method %r is already locked!" % method)
    def locked_method(self, *arg, **kwarg):
        with self._lock:
            return method(self, *arg, **kwarg)
    locked_method.__name__ = '%s(%s)' % ('lock_method', method.__name__)
    locked_method.__is_locked = True
    return locked_method


def make_threadsafe(cls, methodnames, lockfactory):
    init = cls.__init__
    def newinit(self, *arg, **kwarg):
        init(self, *arg, **kwarg)
        self._lock = lockfactory()
    cls.__init__ = newinit

    for methodname in methodnames:
        oldmethod = getattr(cls, methodname)
        newmethod = lock_method(oldmethod)
        setattr(cls, methodname, newmethod)

    return cls


@lock_class(['add','remove'], Lock)
class ClassDecoratorLockedSet(set):

    @lock_method # if you double-lock a method, a TypeError is raised
    def lockedMethod(self):
        print("This section of our code would be thread safe")
        pass


Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 907
Condition Released by Publisher: Thread-26
907 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
119 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
119 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
107 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
107 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 238
Condition Released by Publisher: Thread-26
238 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Co

condition acquired by Thread-24
226 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
226 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
118 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
118 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 998
Condition Released by Publisher: Thread-26
998 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
154 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
154 popped from list by Thread-25
condition released by Thread-25
condition

In [28]:
# Decorator Set 


def locked_method(method):
    """Method decorator. Requires a lock object at self._lock"""
    def newmethod(self, *args, **kwargs):
        with self._lock:
            return method(self, *args, **kwargs)
    return newmethod

class DecoratorLockedSet(set):
    def __init__(self, *args, **kwargs):
        self._lock = Lock()
        super(DecoratorLockedSet, self).__init__(*args, **kwargs)

    @locked_method
    def add(self, *args, **kwargs):
        return super(DecoratorLockedSet, self).add(elem)

    @locked_method
    def remove(self, *args, **kwargs):
        return super(DecoratorLockedSet, self).remove(elem)

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 992
Condition Released by Publisher: Thread-26
992 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
174 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
174 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
44 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
44 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 973
Condition Released by Publisher: Thread-26
973 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Cond

condition acquired by Thread-24
248 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
248 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
22 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
22 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 328
Condition Released by Publisher: Thread-26
328 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
48 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
48 popped from list by Thread-25
condition released by Thread-25
condition acq

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 208
Condition Released by Publisher: Thread-26
208 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
139 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
139 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
238 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
238 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 783
Condition Released by Publisher: Thread-26
783 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Co

condition acquired by Thread-24
237 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
237 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
94 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
94 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 468
Condition Released by Publisher: Thread-26
468 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
136 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
136 popped from list by Thread-25
condition released by Thread-25
condition a

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 668
Condition Released by Publisher: Thread-26
668 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
86 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
86 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
85 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
85 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 647
Condition Released by Publisher: Thread-26
647 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condit

...

condition acquired by Thread-24
217 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
217 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
125 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
125 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 751
Condition Released by Publisher: Thread-26
751 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
126 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
126 popped from list by Thread-25
condition released by Thread-25
condition

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 590
Condition Released by Publisher: Thread-26
590 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
155 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
155 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
223 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
223 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 979
Condition Released by Publisher: Thread-26
979 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Co

condition acquired by Thread-24
4 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
4 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
29 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
29 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 681
Condition Released by Publisher: Thread-26
681 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
180 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
180 popped from list by Thread-25
condition released by Thread-25
condition acqui

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 837
Condition Released by Publisher: Thread-26
837 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
56 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
56 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
157 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
157 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 217
Condition Released by Publisher: Thread-26
217 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Cond

condition acquired by Thread-24
238 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
238 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
212 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
212 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 397
Condition Released by Publisher: Thread-26
397 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27


In [29]:
#Web Crawler 


from urllib.request import Request, urlopen, urljoin, URLError
from urllib.parse import urlparse
import time
import threading
import queue
from bs4 import BeautifulSoup
import ssl

class Crawler(threading.Thread):

  def __init__(self, baseUrl, linksToCrawl, haveVisited, errorLinks, urlLock):
    threading.Thread.__init__(self);
    print("Web Crawler Worker Started: {}".format(threading.current_thread()))
    self.linksToCrawl = linksToCrawl
    self.haveVisited = haveVisited
    self.baseUrl = baseUrl
    self.urlLock = urlLock
    self.errorLinks = errorLinks

  def run(self):
    # We create this context so that we can crawl 
    # https sites
    myssl = ssl.create_default_context();
    myssl.check_hostname=False
    myssl.verify_mode=ssl.CERT_NONE
    # process all the links in our queue
    while True:
      
      self.urlLock.acquire()
      print("Queue Size: {}".format(self.linksToCrawl.qsize()))
      link = self.linksToCrawl.get()
      self.urlLock.release()
      # have we reached the end of our queue?
      if link is None:
        break

      # Have we visited this link already?
      if (link in self.haveVisited):
        print("Already Visited: {}".format(link))
        break
      
      try:
        link = urljoin(self.baseUrl, link)
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        response = urlopen(req, context=myssl)

        print("Url {} Crawled with Status: {}".format(response.geturl(), response.getcode()))
        
        soup = BeautifulSoup(response.read(), "html.parser")
        
        for atag in soup.find_all('a'):
          if (atag.get('href') not in self.haveVisited) and (urlparse(link).netloc == 'tutorialedge.net'):
            self.linksToCrawl.put(atag.get('href'))
          else :
            print("{} already visited or not part of website".format(atag.get('href')))

        print("Adding {} to crawled list".format(link))
        self.haveVisited.append(link)
        
      except URLError as e:
        print("URL {} threw this error when trying to parse: {}".format(link, e.reason))
        self.errorLinks.append(link)
      finally:
        self.linksToCrawl.task_done()
  
    

def main():
  print("Starting our Web Crawler")
  baseUrl = input("Website > ")
  numberOfThreads = input("No Threads > ")

  linksToCrawl = queue.Queue()
  urlLock = threading.Lock()
  linksToCrawl.put(baseUrl)
  haveVisited = []
  crawlers = []
  errorLinks = []

  for i in range(int(numberOfThreads)):
    crawler = Crawler(baseUrl, linksToCrawl, haveVisited, errorLinks, urlLock)
    crawler.start()
    crawlers.append(crawler)

  for crawler in crawlers:
    crawler.join()

  print("Total Number of Pages Visited {}".format(len(haveVisited)))
  print("Total Number of Pages with Errors {}".format(len(errorLinks)))


if __name__ == '__main__':
  main()





Starting our Web Crawler
condition acquired by Thread-24
26 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
26 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
136 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
136 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 65
Condition Released by Publisher: Thread-26
65 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
95 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
95 popped from list by Thread-25
condition released by 

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 352
Condition Released by Publisher: Thread-26
352 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
171 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
171 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
127 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
127 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 815
Condition Released by Publisher: Thread-26
815 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Co

condition acquired by Thread-24
72 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
72 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
133 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
133 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 3
Condition Released by Publisher: Thread-26
3 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
27 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
27 popped from list by Thread-25
condition released by Thread-25
condition acquire

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 468
Condition Released by Publisher: Thread-26
468 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
40 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
40 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
122 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
122 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 166
Condition Released by Publisher: Thread-26
166 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Cond

condition acquired by Thread-24
237 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
237 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
213 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
213 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 502
Condition Released by Publisher: Thread-26
502 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
248 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
248 popped from list by Thread-25
condition released by Thread-25
condition

...

condition acquired by Thread-24
59 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
59 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
130 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
130 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 237
Condition Released by Publisher: Thread-26
237 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
99 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
99 popped from list by Thread-25
condition released by Thread-25
condition acq

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 961
Condition Released by Publisher: Thread-26
961 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
39 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
39 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
67 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
67 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 979
Condition Released by Publisher: Thread-26
979 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condit

KeyboardInterrupt: 

Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 584
Condition Released by Publisher: Thread-26
584 Popped from list by Consumer: Thread-27
Consumer Thread-27 Releasing Condition
Condition Acquired by Consumer: Thread-27
Condition Wait by Thread-27
condition acquired by Thread-24
226 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
226 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
168 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
168 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 398
Condition Released by Publisher: Thread-26
398 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Co

condition acquired by Thread-24
53 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
53 popped from list by Thread-25
condition released by Thread-25
condition acquired by Thread-25
condition wait by Thread-25
condition acquired by Thread-22
89 appended to list by Thread-22
condition notified by Thread-22
condition released by Thread-22
89 popped from list by Thread-23
condition released by Thread-23
condition acquired by Thread-23
condition wait by Thread-23
Condition Acquired by Publisher: Thread-26
Publisher Thread-26 appending to array: 36
Condition Released by Publisher: Thread-26
36 Popped from list by Consumer: Thread-28
Consumer Thread-28 Releasing Condition
Condition Acquired by Consumer: Thread-28
Condition Wait by Thread-28
condition acquired by Thread-24
115 appended to list by Thread-24
condition notified by Thread-24
condition released by Thread-24
115 popped from list by Thread-25
condition released by Thread-25
condition acqui

In [ ]:
# FOR ADVANCED PROGRAMMING REFER RAO - CONCURRENCY IN PYTHON EXAMPLES, PERFORMANCE TUNING SAMPLES SECTION 